# Linear regression  
## FINDING THE OVERALL RATING OF A FOOTBALL PLAYER WITH GIVEN FEATURES

In [ ]:
import pandas as pd
from sklearn import svm
import sqlite3
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
%reset out

## CONNECT TO THE DATABASE

In [ ]:
conn = sqlite3.connect(r'C:\D\New_Stuff\soccer\database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)

## LET'S TAKE A LOOK AT THE DATA

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df.dtypes

## SINCE THIS IS A REGRESSION PROBLEM , LET US CONVERT ALL THE STRINGS TO FLOAT BY ASSIGNING THEM A NUMBER COREESPONDING TO THE SKILL LEVEL

In [ ]:
df['preferred_foot'] = df['preferred_foot'].replace('right',0).replace('left',1)
df[['attacking_work_rate','defensive_work_rate']] = df[['attacking_work_rate','defensive_work_rate']].replace('high',3).replace('medium',2).replace('low',1)

In [ ]:
for i in df[['attacking_work_rate','defensive_work_rate']]:
    list1 = set()
    list2 = [1,2,3,4,5,6,7,8,9,0]
    list3 = ['1','2','3','4','5','6','7','8','9','0']
    for x in df[i]:
        if x in list2:
            continue
        else:
            list1.add(x)
    print(list1)
    for x in list1:
        if x == 'norm':
            df[i] = df[i].replace('norm',2)
        elif x == 'ormal':
            df[i] = df[i].replace('ormal',2)
        elif x in list3:
            x = float(x)
            if x > 6:
                df[i] = df[i].replace(x,3)
            elif x > 3 and x < 7:
                df[i] = df[i].replace(x,2)
            elif x > 0 and x < 4:
                df[i] = df[i].replace(x,1)
            else :
                df[i] = df[i].replace(x,0)
            
        else:
            df[i] = df[i].replace(x,0)
    df[i] = df[i].astype(float)


In [ ]:
df.shape

## TAKE CARE OF THE MISSING VALUES

In [ ]:
df = df.dropna()

In [ ]:
df.shape

## SELECT ALL THE FEATURES WE WANT 
### WE HAVE LEFT OUT  'id', 'player_fifa_api_id', 'player_api_id', 'date' Because they are not relevant for finding the overall rating of a player

In [ ]:
features = ['potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

## SELECT THE TARGET VARIABLE

In [ ]:
target = ['overall_rating']

In [ ]:
X  = df[features]

In [ ]:
y = df[target]

In [ ]:
y.head(5)

## SPLIT THE DATA INTO TRAINING AND TEST SET

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=234)

## FIRST, WE WILL FIT A  LINEAR REGRESSION MODEL USING THE TRAINING SET 

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

## LET US PREDICT VALUES USING THE TEST SET

In [ ]:
y_prediction = regressor.predict(X_test)
y_prediction

## LETS SEE THE MARGIN OF ERROR USING ROOT MEAN SQUARED ERROR 
### THE LOWER THE BETTER

In [ ]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
print(RMSE)

## LETS USE THE DecisionTreeRegressor TO SEE IF WE CAN GET A BETTER RMSE SCORE

In [ ]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, y_train)

In [ ]:
y_prediction = regressor.predict(X_test)
y_prediction

In [ ]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
print(RMSE)

## THE DecisionTreeRegressor GOT A MUCH BETTER SCORE FOR THIS PARTICULAR DATASET